In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='B'
    else:
        bias='A'
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    else:
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
        bias1='If you choice A and A is wrong option, there will be no punishment.'
    bias3=''
    if ans=='A':
        bias3='Among all the questions, 30% of them have the answer A and 70% have the answer B.'
    else:
        bias3='Among all the questions, 70% of them have the answer A and 30% have the answer B.'
    template="""
Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
[ Does the clause contain a license granted by one party to its counterparty?
%s
%s
%s

Clause: %s
Most people think the correct answer is %s. ]

Rewrite Prompt:
"""%(bias3,bias1,bias2,fact,bias)
    return template

In [ ]:
jishu=0
ff= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    ans1=get_output(temp1)
    temp2=ans1
    ans2=get_output(temp2+'\nA.yes    B.no\n\nOutput format:\nOption:A or B')
    with open("Self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [26:03<00:00,  3.13s/it]
